In [1]:
from pathlib import Path
import pandas as pd

from papyrus_scripts.reader import read_protein_descriptors
import papyrus_scripts

import Bio
import Bio.SeqIO as Bio_SeqIO
import rich
import rich_msa

import prodec

In [2]:
from prodec import *
pdescs = ProteinDescriptors()
print(pdescs.available_descriptors)

['ADFQ', 'BLOSUM', 'c-scales', 'CBFQ', 'CDFQ', 'Combined descriptors', 'Contact energies', 'CUFQ', 'DPPS', 'E-scale', 'FASGAI', 'G-scales', 'GH-scale', 'GRID tscore', 'HESH', 'HPI', 'HSEHPCSV', 'Independent descriptors', 'ISA-ECI', 'Kidera', 'MS-WHIM', 'P-scale', 'PhysChem', 'ProtFP hash', 'ProtFP PCA', 'PSM', 'QCP', 'Raychaudhury', 'Sneath', 'SSIA AM1', 'SSIA DFT', 'SSIA HF', 'SSIA PM3', 'STscale', 'SVEEVA', 'SVGER', 'SVHEHS', 'SVMW', 'SVRDF', 'SVRG', 'SVWG', 'SZOTT', 'Tscale', 'V-scale', 'VARIMAX', 'VHSE', 'VHSEH', 'VSGETAWAY', 'VSTPV', 'VSTV', 'VSW', 'VTSA', 'Zscale binary', 'Zscale Hellberg', 'Zscale Jonsson', 'Zscale Sandberg', 'Zscale Sjöström', 'Zscale van Westen']


In [2]:
# Set paths 
HERE = Path(_dh[-1])
DATA =  HERE.parents[0] / "PCM_modelling"
PAPYRUS_VERSION = '05.5'

In [3]:
# Define uniprot name and accession codes for targets of interest (3ddpd set)
targets = {'5ht1b_human': 'P28222', 
           '5ht2b_human': 'P41595',
           'aa1r_human': 'P30542',
           'aa2ar_human': 'P29274',
           'acm1_human': 'P11229',
           'acm2_human': 'P08172',
           'acm4_human': 'P08173',
           'adrb2_human': 'P07550',
           'agtr1_human': 'P30556',
           'ccr5_human': 'P51681',
           'cnr1_human': 'P21554',
           'cxcr4_human': 'P61073',
           'drd3_human': 'P35462',
           'ednrb_human': 'P24530',
           'ffar1_human': 'O14842',
           'hrh1_human': 'P35367',
           'lpar1_human': 'Q92633',
           'oprd_human': 'P41143',
           'oprk_human': 'P41145',
           'oprx_human': 'P41146',
           'ox1r_human': 'O43613',
           'ox2r_human': 'O43614',
           'p2ry1_human': 'P47900',
           'p2y12_human': 'Q9H244',
           'par1_human': 'P25116',
           's1pr1_human': 'P21453'
             }

# Define target IDs for mapping in Papyrus
target_ids = [f'{accession}_WT' for accession in targets.values()]

### Multiple sequence alignment 

In [4]:
# Read target sequences 
protein_data = papyrus_scripts.read_protein_set(version=PAPYRUS_VERSION, source_path=DATA)
# Filter protein data for our targets of interest based on accession code
targets = protein_data[protein_data.target_id.isin(target_ids)]
targets.head()

,target_id,HGNC_symbol,UniProtID,Status,Organism,Classification,Length,Sequence
15,P25116_WT,F2R,PAR1_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,425,MGPRRLLLVAACFSLCGPLLSARTRARRPESKATNATLDPRSFLLR...
40,P28222_WT,HTR1B,5HT1B_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,390,MEEPGAQCAPPPPAGSETWVPQANLSSAPSQNCSAKDYIYQDSISL...
80,P30542_WT,ADORA1,AA1R_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,326,MPPSISAFQAAYIGIEVLIALVSVPGNVLVIWAVKVNQALRDATFC...
81,P29274_WT,ADORA2A,AA2AR_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,412,MPIMGSSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVV...
99,P35462_WT,DRD3,DRD3_HUMAN,reviewed,Homo sapiens (Human),Membrane receptor->Family A G protein-coupled ...,400,MASLSQLSGHLNYTCGAENSTGASQARPHAYYALSYCALILAIVFG...


In [5]:
# Write protein sequences to file
# Create object with sequences and descriptions
records = []
for index, row in targets.reset_index(drop=True).iterrows():
    records.append(
        Bio_SeqIO.SeqRecord(
            seq=Bio.Seq.Seq(row["Sequence"]),
            id=str(index),
            name=row["target_id"],
            description=" ".join([row["UniProtID"], row["Organism"], row["Classification"]]),
        )
    )
sequences_path = Path(DATA / "sequences.fasta")
# Write sequences as .fasta file
_ = Bio_SeqIO.write(records, sequences_path, "fasta")

In [9]:
# Calculate MSA for proteins of interest
!clustalo -i sequences.fasta -t Protein -o msa.fasta --outfmt=fa 

In [10]:
# Parse aligned sequences
alignment_file = "msa.fasta"
aligned_sequences = [str(seq.seq) for seq in Bio.SeqIO.parse(alignment_file, "fasta")]

In [11]:
# Read MSA
msa = Bio.AlignIO.read(alignment_file, "fasta")
viewer = rich_msa.RichAlignment(
    names=[record.description for record in msa],
    sequences=[str(record.seq) for record in msa],
)
# Visualize MSA
panel = rich.panel.Panel(viewer, title="Multiple sequence alignment")
rich.print(panel)

╭────────────────────────────────────────── Multiple sequence alignment ──────────────────────────────────────────╮
│ 0 PAR1_HU…     1  MGP-------------------------------RRLLLVAACFSLCGPLLSARTRARRPESKATNATLDPRSFLLRNPNDKYE--------  │
│ 1 5HT1B_H…     1  --------------------------------------------------------MEEPGAQCAP---------PPPAGSETW--------  │
│ 2 AA1R_HU…     1  --------------------------------------------------------------------------------------------  │
│ 3 AA2AR_H…     1  --------------------------------------------------------------------------------------------  │
│ 4 DRD3_HU…     1  --------------------------------------------------------------------------------MASL--------  │
│ 5 OPRD_HU…     1  ---------------------------------------------------------MEPAPSAGAELQPP---LF-ANASDAY--------  │
│ 6 CCR5_HU…     1  --------------------------------------------------------------------------------MDYQ--------  │
│ 7 ACM1_HU…     1  --------------------------------------------------------------------------------------------  │
│ 8 ACM2_HU…     1  --------------------------------------------------------------------------------------------  │
│ 9 ACM4_HU…     1  ------------------------------------------------------------------[1;

### Protein descriptors

In [17]:
def calculate_protein_descriptor(targets, aligned_sequences, protein_descriptor):
    """
    Calculate protein descriptor of choice for aligned proteins of interest

    Parameters
    ----------
    targets : pandas.DataFrame
        Pandas dataframe with information about targets of interest
    aligned_sequences : list
        List of aligned sequences read from fasta file produced with Clustal Omega
    protein_descriptor : str
        Protein descriptor label as described in ProDEC

    Returns
    -------
    pandas.DataFrame
        Dataset with accession and features for the protein descriptor of interest for the targets in the input
    """
    # Get protein descriptor from ProDEC
    prodec_descriptor = prodec.ProteinDescriptors().get_descriptor(protein_descriptor)

    # Calculate descriptor features for aligned sequences of interest
    protein_features = prodec_descriptor.pandas_get(aligned_sequences)

    # Insert protein labels in the obtained features
    protein_features.insert(0, "target_id", targets.target_id.reset_index(drop=True))

    return protein_features

In [ ]:
protein_features = calculate_protein_descriptor(targets, aligned_sequences, "Zscale van Westen")
protein_features.to_csv('zscale_van_westen.txt', sep='\t')
protein_features.head()